In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
#Import the two files we were provided with into pandas dataframses
data_test = pd.read_csv('fashion-mnist_test.csv')
data_train = pd.read_csv('fashion-mnist_train.csv')

In [3]:
#The data that we have still needs to be split into training and test, Y and X 
test_X=data_test.drop(["label"], axis=1)
test_Y=data_test["label"]
train_X=data_train.drop(["label"], axis=1)
train_Y=data_train["label"]

In [4]:
#Here the data is converted into numpy arrays so it can later be fit into the chosen model
X_train = np.asarray(train_X)
Y_train = np.asarray(train_Y)
X_test = np.asarray(test_X)
Y_test = np.asarray(test_Y)

In [5]:
#Use a funtion to create a model and pass number of layers and neurons. We also have to set initial weights 
def create_model(num_layer, num_neuron):
    model = tf.keras.Sequential()
    kernel_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=2 / np.sqrt(len(X_train) + num_neuron))
    model.add(tf.keras.layers.Dense(num_neuron, input_shape=(28 * 28,), activation="relu", kernel_initializer=kernel_initializer))
    for i in range(num_layer):
        model.add(tf.keras.layers.Dense(num_neuron, activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [6]:
#Now we have to find he hyperparameters that have the highest accuracy. Random Search is used for that. 
#We are also choosing the layers: 1, 2, 3, 4. And also chhosing neurons: 96, 112, 128, 144
num_layer = [1, 2, 3, 4]
num_neuron = [96, 112, 128, 144]
param_dist = dict(num_layer=num_layer, num_neuron=num_neuron)

In [7]:
# Now we have to create an esimator to pass into the random search function. Number of epochs 40 and batch size 500
model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=40, batch_size=500, verbose=0)
random_search = RandomizedSearchCV(estimator=model,param_distributions=param_dist, n_iter=10, cv=3)
random_search.fit(X_train, Y_train)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x00000164E3F57D08>,
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'num_layer': [1, 2, 3, 4],
                                        'num_neuron': [96, 112, 128, 144]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [9]:
print("Highest accuracy is: ",random_search.best_score_," with the use of ", random_search.best_params_)
mean_score = random_search.cv_results_["mean_test_score"]
std_score = random_search.cv_results_["std_test_score"]
params = random_search.cv_results_["params"]
for mean, stdev, param in zip(mean_score, std_score, params):
    print("The accuracy of ",random_search.best_score_," with the use of ",mean_score, std_score, param)

Highest accuracy is:  0.8789666692415873  with the use of  {'num_neuron': 144, 'num_layer': 3}
The accuracy of  0.8789666692415873  with the use of  [0.87395    0.87578332 0.85946667 0.86443333 0.86860001 0.87644998
 0.86216666 0.86528331 0.87896667 0.86113334] [0.00063378 0.00198591 0.00251142 0.00252332 0.00648113 0.00271755
 0.00198089 0.00690656 0.00409966 0.0073467 ] {'num_neuron': 128, 'num_layer': 3}
The accuracy of  0.8789666692415873  with the use of  [0.87395    0.87578332 0.85946667 0.86443333 0.86860001 0.87644998
 0.86216666 0.86528331 0.87896667 0.86113334] [0.00063378 0.00198591 0.00251142 0.00252332 0.00648113 0.00271755
 0.00198089 0.00690656 0.00409966 0.0073467 ] {'num_neuron': 112, 'num_layer': 3}
The accuracy of  0.8789666692415873  with the use of  [0.87395    0.87578332 0.85946667 0.86443333 0.86860001 0.87644998
 0.86216666 0.86528331 0.87896667 0.86113334] [0.00063378 0.00198591 0.00251142 0.00252332 0.00648113 0.00271755
 0.00198089 0.00690656 0.00409966 0.007

In [10]:
#We have to now fit the final model with the best hyperparameters
final_model_hyperpar=create_model(random_search.best_params_["num_layer"], random_search.best_params_["num_neuron"])
final_model_hyperpar.fit(X_train, Y_train)

Train on 60000 samples
60000/60000 [==============================] - 8s 133us/sample - loss: 0.7992 - accuracy: 0.7766


In [13]:
#Evaluate the final model and its final accuracy
final_result = final_model_hyperpar.evaluate(X_test,Y_test, verbose=0)
print("Final accuracy is ", final_result[1])

Final accuracy is  0.832
